<a href="https://colab.research.google.com/github/LakshmanYES/My-School-Projects/blob/main/Sai_Lakshman_Ethakatla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1908GDMdrhDN7sTaI_FelSHxbwcNM1EzR -O /content/drive/MyDrive/data/goodreads_reviews_children.json.gz
!gunzip /content/drive/MyDrive/data/goodreads_reviews_children.json.gz 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1908GDMdrhDN7sTaI_FelSHxbwcNM1EzR
To: /content/drive/MyDrive/data/goodreads_reviews_children.json.gz
100% 172M/172M [00:01<00:00, 160MB/s]


In [ ]:
# Spark init
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
#To load The data
df_data_books = spark.read.json("/content/drive/MyDrive/data/goodreads_reviews_children.json")
df_data_books.show()

+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
| book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|
+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23310161|Tue Nov 17 11:37:...|Tue Nov 17 11:38:...|         0|      7|     4|                    |f4b4b050f4be00e92...|Fun sequel to the...|                    |8842281e1d1347389...|
|17290220|Sat Nov 08 08:54:...|Wed Jan 25 13:56:...|         0|      4|     5|Tue Jan 24 00:00:...|22d424a2b0057b18f...|One of my favorit...|                    |8842281e1d1347389...|
| 6954929|Thu Oct 23 13:46:...|Thu Oct 23 13:47:...|         1|      6|     5|  

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, IntegerType, DateType
df_data = df_data_books \
    .withColumn('book_id', col('book_id').cast(IntegerType())) 

In [ ]:
df_data_books = df_data_books.withColumn("book_id",df_data_books.book_id.cast(IntegerType()))
df_data_books.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- n_comments: long (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- read_at: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- user_id: string (nullable = true)



In [ ]:
df_data_books_filtered = df_data_books[df_data_books['n_votes'] == 0]
df_data_books_filtered.show()
df_data_books_filtered.count()

+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
| book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|
+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   10365|Thu Jan 03 10:24:...|Wed Mar 22 11:45:...|         0|      0|     5|Fri Apr 01 00:00:...|b953295db05c92394...|    Loved this book.|                    |8842281e1d1347389...|
|23302416|Wed Jan 21 18:40:...|Wed Oct 26 03:44:...|         0|      0|     5|                    |84c0936a0f9868f38...|I read this book ...|                    |7504b2aee1ecb5b28...|
|17381987|Fri Sep 11 10:15:...|Sat Nov 21 14:01:...|         0|      0|     5|  

602872

In [ ]:
import pyspark.sql.functions as func

In [ ]:
#Average character length of review text
average_character_length = df_data_books.agg(func.avg(func.length("review_text"))).collect()[0][0]
print('Average character length of review text:',average_character_length)

Average character length of review text: 421.9224994555156


In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="user_id", outputCol="user_index")
df_data_books_indexed = indexer.fit(df_data_books).transform(df_data_books)
df_data_books_indexed.show()
df_data_books_indexed.count()

+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|user_index|
+--------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|23310161|Tue Nov 17 11:37:...|Tue Nov 17 11:38:...|         0|      7|     4|                    |f4b4b050f4be00e92...|Fun sequel to the...|                    |8842281e1d1347389...|   10129.0|
|17290220|Sat Nov 08 08:54:...|Wed Jan 25 13:56:...|         0|      4|     5|Tue Jan 24 00:00:...|22d424a2b0057b18f...|One of my favorit...|                    |8842281e1d1347389...|   10129.0|
| 6954929|Thu Oct 23 13:4

734640

In [ ]:

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
#Training the data
(training, test) = df_data_books_indexed.randomSplit([0.8, 0.2], seed = 123)
print (training.first())

Row(book_id=5, date_added='Fri Apr 04 06:26:04 -0700 2008', date_updated='Fri Aug 29 06:53:00 -0700 2014', n_comments=0, n_votes=0, rating=5, read_at='Thu Aug 28 00:00:00 -0700 2014', review_id='263bc6ff81c9202bff11393d54f67208', review_text='Rowling gets noticeably better as a writer as her series continues. There is a lot of good stuff in here, particularly the Sirius Black and Professor Lupin characterizations. Even leading up to the last couple of pages, you really think you know who these guys are. Well, not everything is what it seems, right? \n Heck, you even feel for Buckbeak, a hippogriff set up by Draco Malfoy to be too dangerous to be kept alive. And last but not least, the time turner. As I have read this series before, nothing surprised me, of course, but I do remember being surprised about the time turner. Time travel is such a fascinating concept, and can cause one to think about some crazy stuff. What would I do with the time turner if I had one? \n On to book 4!', star

In [ ]:
#Creating ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="user_index", itemCol="book_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
print ("Total users in training data:", model.userFactors.count())
print ("Total books in training data:", model.itemFactors.count())

Total users in training data: 83195
Total books in training data: 111160


In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [ ]:
predictions.show()

+-------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+
|book_id|          date_added|        date_updated|n_comments|n_votes|rating|             read_at|           review_id|         review_text|          started_at|             user_id|user_index| prediction|
+-------+--------------------+--------------------+----------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----------+
|  14570|Thu Sep 18 13:41:...|Thu Sep 18 13:42:...|         0|      0|     5|                    |e45398f6cb805ae22...|Really cute class...|                    |6183fba9ea5f709f8...|    7331.0| -5.9512305|
|  14570|Fri Jun 01 15:52:...|Thu Jul 26 17:18:...|         0|      0|     4|Thu Jul 26 00:00:...|52860a12c66d6a4f9...|It was a sweet an...|Thu Jun 21 00:00:...|df3d07d79ccd093

In [ ]:
#Generating RMSE value
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error of the ALS model = " + str(rmse))

Root-mean-square error of the ALS model = 3.3925669252239197


In [ ]:
users = df_data_books_indexed.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 5)

userSubsetRecs.show(truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------+
|user_index|recommendations                                                                                                    |
+----------+-------------------------------------------------------------------------------------------------------------------+
|62706     |[{16238459, 7.2577124}, {13548546, 6.4456367}, {15958035, 6.3328094}, {18405814, 6.0609283}, {24748294, 5.8095713}]|
|51869     |[{29781406, 18.599735}, {28580791, 16.600353}, {33960518, 16.307196}, {25619537, 15.74214}, {13558060, 15.698348}] |
|10129     |[{13548546, 49.69863}, {16238459, 44.94}, {16713133, 43.445107}, {12480717, 43.193115}, {298498, 42.428764}]       |
+----------+-------------------------------------------------------------------------------------------------------------------+

